In [29]:
import pandas as pd

df = pd.read_csv('cleaned_movies.csv')

In [30]:
df.head()

,movie_id,title,tags
0,19995,Avatar,"22nd century, paraplegic marine dispatch moon ..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believe dead, come back..."
2,206647,Spectre,cryptic message bond’s past sends trail uncove...
3,49026,The Dark Knight Rises,"follow death district attorney harvey dent, ba..."
4,49529,John Carter,"john carter war-weary, former military captain..."


hmare paas ab cleaned dataset hai jisme teen col h movie_id for poster fetching , title for showing the movies , tags used for recommendation

# Model Training 

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features = 5000)

hmne isme countvectorizer ki jgh tfidfvectorizer use kiya coz in that we are only focusing on no. of times a word occur but in this we are also focusing that is this word rare

In [32]:
tfidf_matrix = tfidf.fit_transform(df['tags'])       # fit -> vocabulary of words banayega , transform -> numeric vector me convert karega hr tag ko
tfidf_matrix.shape

(4806, 5000)

ye ek sparse matrix bna rha h (most entries zero hogi kyuki most of the wors hr movie m nhi aye honge) jo ki hm cosine similarity m use kr skte h 

In [33]:
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 162412 stored elements and shape (4806, 5000)>

In [34]:
tfidf.get_feature_names_out()

array(['000', '007', '10', ..., 'zoo', 'zooeydeschanel', 'zoëkravitz'],
      shape=(5000,), dtype=object)

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf_matrix)

In [36]:
cos_sim

array([[1.        , 0.01681646, 0.02570707, ..., 0.02051888, 0.00574195,
        0.        ],
       [0.01681646, 1.        , 0.05508901, ..., 0.01462689, 0.        ,
        0.        ],
       [0.02570707, 0.05508901, 1.        , ..., 0.01533068, 0.        ,
        0.        ],
       ...,
       [0.02051888, 0.01462689, 0.01533068, ..., 1.        , 0.02826751,
        0.02887436],
       [0.00574195, 0.        , 0.        , ..., 0.02826751, 1.        ,
        0.03388662],
       [0.        , 0.        , 0.        , ..., 0.02887436, 0.03388662,
        1.        ]], shape=(4806, 4806))

yha pr hmne cosine similarity use kiya (isme ek movie ka dusri movie k saath cosine distance calculate) jisse hm movie recommend kr paye <br>
higher cos_sim indicates movie is similar to another movie

In [37]:
import joblib
joblib.dump(df, 'movies_df.joblib')

['movies_df.joblib']

In [38]:
from scipy.sparse import csr_matrix, save_npz
sparse_cos_sim = csr_matrix(cos_sim)
save_npz('cos_sim_sparse.npz', sparse_cos_sim)

In [39]:
from scipy.sparse import load_npz

cos_sim = load_npz('cos_sim_sparse.npz')
df = joblib.load('movies_df.joblib')


In [40]:
def recommend(movie):
    index = df[df['title'] == movie].index[0]       # isse hm movie ka hmare data m index nikaal rhe h
    distances = cos_sim[index]                      # isse hm us movie k saath sare movies k cosine similarity nikaal rhe h
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]  
    # hmne distances ko sort krke top 5 movies ko rkha hai 

    for i in movies_list:
        print(df.iloc[i[0]].title)      # iloc se hm index k basis pr movie title nikaal rhe h  , i -> tuple h ( index , cosine similarity value)


hm distances ko sort kr rhe h enumerate ka use krke kyuki agr hmne iske bina kiya to hm index pos loose kr denge phr correct movie ko predict nhi kr payenge , enumerate key value pair bna rha hai jisse indexx pos ki help se hm movie ko recommend kr payege <br>
key ka use hmne distances k basis pe sort kre uske liye rkha hai wrna index k basis pe kr rha tha <br>
reverse = True -> for descending 

In [41]:
recommend('Avatar')

In [42]:
recommend('Batman Begins')

In [43]:
recommend('Superman Returns')